In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from keras.optimizers import Adam, RMSprop
from tensorflow.python.keras import regularizers
from keras import backend as K
from keras.layers import Masking
from keras.utils import to_categorical
import keras

Using TensorFlow backend.


### Data preprocessing

In [2]:
df = pd.read_csv("ml-latest-small/ratings.csv")

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
df.index = df.userId
del df["userId"]

In [9]:
# Creation of the rating matrix in which the columns are the item id and the rows the user id 
X = np.empty(shape = (df.index.unique().shape[0],df["movieId"].unique().shape[0]))
X= pd.DataFrame(X,columns=df["movieId"].unique(),index =df.index.unique() )

for u in df.index.unique():
    for i in df["movieId"].unique():
        v = df.loc[(df["movieId"]==i) & (df.index==u)]["rating"]
        if v.empty:
            X[i][u]= np.nan
        else:
            X[i][u] = v.values[0]

In [11]:
X.head()

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,4.0,5.0,5.0,3.0,5.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# New loss function which does not take into account the entries with the value nan (or 99)
X.fillna(99,inplace=True)

def masked_mse(y_true, y_pred):
    mask_true = K.cast(K.not_equal(y_true, 99), K.floatx())
    masked_squared_error = K.square(mask_true * (y_true - y_pred))
    masked_mse = K.sum(masked_squared_error, axis=-1) / K.maximum(K.sum(mask_true, axis=-1), 1)
    return masked_mse

### Auto Encoder 1

In [17]:
input_layer = Input(shape=(X.shape[1],))
encoded1 = Dense(500, activation="relu",kernel_regularizer=regularizers.l2(0.0005))(input_layer)
decoded1 = Dense(X.shape[1], activation="relu",kernel_regularizer=regularizers.l2(0.0005))(encoded1)
AE1 = Model(input_layer, decoded1)
AE1.compile(optimizer = Adam(lr=0.0001), loss=masked_mse)

callback_AE1 = [keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.3, patience=60,verbose=1),\
                    keras.callbacks.ModelCheckpoint(filepath='AE1.h5',monitor='loss',save_best_only=True,\
                                                  save_weights_only=True,mode='min',verbose=1)]

In [18]:
AE1.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 9724)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               4862500   
_________________________________________________________________
dense_6 (Dense)              (None, 9724)              4871724   
Total params: 9,734,224
Trainable params: 9,734,224
Non-trainable params: 0
_________________________________________________________________


In [22]:
Hist_AE1 = AE1.fit(X, X,
                  epochs=1000,
                  batch_size=256,
                  callbacks=callback_AE1,
                  verbose = 1)

Epoch 1/1000
610/610 [==============================] - 2s 3ms/step - loss: 187.1587

Epoch 00001: loss improved from inf to 187.15870, saving model to AE1.h5
Epoch 2/1000
610/610 [==============================] - 1s 1ms/step - loss: 20.4912

Epoch 00002: loss improved from 187.15870 to 20.49122, saving model to AE1.h5
Epoch 3/1000
610/610 [==============================] - ETA: 0s - loss: 12.02 - 1s 1ms/step - loss: 12.0642

Epoch 00003: loss improved from 20.49122 to 12.06421, saving model to AE1.h5
Epoch 4/1000
610/610 [==============================] - 1s 1ms/step - loss: 11.7224

Epoch 00004: loss improved from 12.06421 to 11.72244, saving model to AE1.h5
Epoch 5/1000
610/610 [==============================] - 1s 1ms/step - loss: 11.2092

Epoch 00005: loss improved from 11.72244 to 11.20915, saving model to AE1.h5
Epoch 6/1000
610/610 [==============================] - 1s 1ms/step - loss: 10.7199

Epoch 00006: loss improved from 11.20915 to 10.71993, saving model to AE1.h5
Epoch 

610/610 [==============================] - 1s 1ms/step - loss: 7.7429

Epoch 00052: loss improved from 7.75778 to 7.74295, saving model to AE1.h5
Epoch 53/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.7303

Epoch 00053: loss improved from 7.74295 to 7.73027, saving model to AE1.h5
Epoch 54/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.7327

Epoch 00054: loss did not improve from 7.73027
Epoch 55/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.7175

Epoch 00055: loss improved from 7.73027 to 7.71750, saving model to AE1.h5
Epoch 56/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.7136

Epoch 00056: loss improved from 7.71750 to 7.71356, saving model to AE1.h5
Epoch 57/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.6890

Epoch 00057: loss improved from 7.71356 to 7.68901, saving model to AE1.h5
Epoch 58/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.682

Epoch 158/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.3260

Epoch 00158: loss improved from 7.32714 to 7.32602, saving model to AE1.h5
Epoch 159/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.3261

Epoch 00159: loss did not improve from 7.32602
Epoch 160/1000
610/610 [==============================] - 1s 2ms/step - loss: 7.3292

Epoch 00160: loss did not improve from 7.32602
Epoch 161/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.3305

Epoch 00161: loss did not improve from 7.32602
Epoch 162/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.3350

Epoch 00162: loss did not improve from 7.32602
Epoch 163/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.3304

Epoch 00163: loss did not improve from 7.32602
Epoch 164/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.3327

Epoch 00164: loss did not improve from 7.32602
Epoch 165/1000
610/610 [=================

610/610 [==============================] - 1s 2ms/step - loss: 7.2484

Epoch 00269: loss did not improve from 7.24335
Epoch 270/1000
610/610 [==============================] - 1s 2ms/step - loss: 7.2459

Epoch 00270: loss did not improve from 7.24335
Epoch 271/1000
610/610 [==============================] - 1s 2ms/step - loss: 7.2460

Epoch 00271: loss did not improve from 7.24335
Epoch 272/1000
610/610 [==============================] - 1s 2ms/step - loss: 7.2453

Epoch 00272: loss did not improve from 7.24335
Epoch 273/1000
610/610 [==============================] - 1s 2ms/step - loss: 7.2412

Epoch 00273: loss improved from 7.24335 to 7.24121, saving model to AE1.h5
Epoch 274/1000
610/610 [==============================] - 1s 2ms/step - loss: 7.2397

Epoch 00274: loss improved from 7.24121 to 7.23971, saving model to AE1.h5
Epoch 275/1000
610/610 [==============================] - 1s 2ms/step - loss: 7.2395

Epoch 00275: loss improved from 7.23971 to 7.23951, saving model to AE1.h5


610/610 [==============================] - 1s 2ms/step - loss: 7.2084

Epoch 00325: loss improved from 7.21040 to 7.20840, saving model to AE1.h5
Epoch 326/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.2081

Epoch 00326: loss improved from 7.20840 to 7.20812, saving model to AE1.h5
Epoch 327/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.2073

Epoch 00327: loss improved from 7.20812 to 7.20729, saving model to AE1.h5
Epoch 328/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.2064

Epoch 00328: loss improved from 7.20729 to 7.20640, saving model to AE1.h5
Epoch 329/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.2064

Epoch 00329: loss improved from 7.20640 to 7.20638, saving model to AE1.h5
Epoch 330/1000
610/610 [==============================] - 1s 1ms/step - loss: 7.2057

Epoch 00330: loss improved from 7.20638 to 7.20565, saving model to AE1.h5
Epoch 331/1000
610/610 [=========================

...


Epoch 00544: loss improved from 6.96946 to 6.96713, saving model to AE1.h5
Epoch 545/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.9701

Epoch 00545: loss did not improve from 6.96713
Epoch 546/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.9689

Epoch 00546: loss did not improve from 6.96713
Epoch 547/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.9649

Epoch 00547: loss improved from 6.96713 to 6.96491, saving model to AE1.h5
Epoch 548/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.9664

Epoch 00548: loss did not improve from 6.96491
Epoch 549/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.9641

Epoch 00549: loss improved from 6.96491 to 6.96414, saving model to AE1.h5
Epoch 550/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.9652

Epoch 00550: loss did not improve from 6.96414
Epoch 551/1000
610/610 [==============================] - 1s 2ms/step 

Epoch 654/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7707

Epoch 00654: loss improved from 6.77407 to 6.77073, saving model to AE1.h5
Epoch 655/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7689

Epoch 00655: loss improved from 6.77073 to 6.76888, saving model to AE1.h5
Epoch 656/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7688

Epoch 00656: loss improved from 6.76888 to 6.76884, saving model to AE1.h5
Epoch 657/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7714

Epoch 00657: loss did not improve from 6.76884
Epoch 658/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7684

Epoch 00658: loss improved from 6.76884 to 6.76838, saving model to AE1.h5
Epoch 659/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7679

Epoch 00659: loss improved from 6.76838 to 6.76790, saving model to AE1.h5
Epoch 660/1000
610/610 [==============================] - 1s 2

610/610 [==============================] - 1s 2ms/step - loss: 6.7089

Epoch 00708: loss improved from 6.70907 to 6.70886, saving model to AE1.h5
Epoch 709/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7108

Epoch 00709: loss did not improve from 6.70886
Epoch 710/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7086

Epoch 00710: loss improved from 6.70886 to 6.70864, saving model to AE1.h5
Epoch 711/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7086

Epoch 00711: loss improved from 6.70864 to 6.70857, saving model to AE1.h5
Epoch 712/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7070

Epoch 00712: loss improved from 6.70857 to 6.70703, saving model to AE1.h5
Epoch 713/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.7023

Epoch 00713: loss improved from 6.70703 to 6.70229, saving model to AE1.h5
Epoch 714/1000
610/610 [==============================] - 1s 2ms/step - loss:

610/610 [==============================] - 1s 2ms/step - loss: 6.6088

Epoch 00821: loss improved from 6.60910 to 6.60882, saving model to AE1.h5
Epoch 822/1000
610/610 [==============================] - 1s 1ms/step - loss: 6.6087

Epoch 00822: loss improved from 6.60882 to 6.60866, saving model to AE1.h5
Epoch 823/1000
610/610 [==============================] - 1s 1ms/step - loss: 6.6081

Epoch 00823: loss improved from 6.60866 to 6.60811, saving model to AE1.h5
Epoch 824/1000
610/610 [==============================] - 1s 1ms/step - loss: 6.6077

Epoch 00824: loss improved from 6.60811 to 6.60773, saving model to AE1.h5
Epoch 825/1000
610/610 [==============================] - 1s 1ms/step - loss: 6.6067

Epoch 00825: loss improved from 6.60773 to 6.60669, saving model to AE1.h5
Epoch 826/1000
610/610 [==============================] - 1s 1ms/step - loss: 6.6072

Epoch 00826: loss did not improve from 6.60669
Epoch 827/1000
610/610 [==============================] - 1s 1ms/step - loss:

Epoch 933/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.4874

Epoch 00933: loss did not improve from 6.48595
Epoch 934/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.4862

Epoch 00934: loss did not improve from 6.48595
Epoch 935/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.4857

Epoch 00935: loss improved from 6.48595 to 6.48572, saving model to AE1.h5
Epoch 936/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.4911

Epoch 00936: loss did not improve from 6.48572
Epoch 937/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.4866

Epoch 00937: loss did not improve from 6.48572
Epoch 938/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.4857

Epoch 00938: loss improved from 6.48572 to 6.48566, saving model to AE1.h5
Epoch 939/1000
610/610 [==============================] - 1s 2ms/step - loss: 6.4880

Epoch 00939: loss did not improve from 6.48566
Epoch 940/100

In [23]:
X_recons = AE1.predict(X)
X_recons=pd.DataFrame(X_recons, index=X.index, columns=X.columns)

In [26]:
X_recons.head(10)

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,4.733685,0.0,4.210973,4.036763,0.0,3.879346,0.0,...,3.432660,3.987811,0.0,0.0,4.035812,0.0,5.126738,3.419458,4.002829,0.0
2,0.0,0.0,0.0,3.950044,0.0,3.512636,3.369169,0.0,3.236513,0.0,...,2.862684,3.328640,0.0,0.0,3.367463,0.0,4.278775,2.852716,3.341533,0.0
3,0.0,0.0,0.0,3.757587,0.0,3.341128,3.205212,0.0,3.078637,0.0,...,2.722702,3.166751,0.0,0.0,3.203320,0.0,4.070521,2.713528,3.179122,0.0
4,0.0,0.0,0.0,3.735191,0.0,3.321170,3.186133,0.0,3.060265,0.0,...,2.706412,3.147912,0.0,0.0,3.184219,0.0,4.046286,2.697331,3.160223,0.0
5,0.0,0.0,0.0,3.927286,0.0,3.492355,3.349781,0.0,3.217844,0.0,...,2.846131,3.309496,0.0,0.0,3.348053,0.0,4.254149,2.836257,3.322327,0.0
6,0.0,0.0,0.0,4.261432,0.0,3.790127,3.634444,0.0,3.491949,0.0,...,3.089170,3.590568,0.0,0.0,3.633038,0.0,4.615721,3.077917,3.604305,0.0
7,0.0,0.0,0.0,3.544600,0.0,3.151326,3.023766,0.0,2.903921,0.0,...,2.567788,2.987594,0.0,0.0,3.021669,0.0,3.840052,2.559493,2.999388,0.0
8,0.0,0.0,0.0,3.843308,0.0,3.417518,3.278239,0.0,3.148955,0.0,...,2.785050,3.238856,0.0,0.0,3.276429,0.0,4.163278,2.775523,3.251460,0.0
9,0.0,0.0,0.0,3.721959,0.0,3.309379,3.174860,0.0,3.049411,0.0,...,2.696788,3.136782,0.0,0.0,3.172934,0.0,4.031969,2.687762,3.149057,0.0


### Deep Auto Encoder 1

In [29]:
input_data2 = Input(shape=(X.shape[1],))
encoded2 = Dense(3000, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(input_data2)
encoded2 = Dense(1000, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(encoded2)
encoded2 = Dense(500, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(encoded2)
decoded2 = Dense(1000, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(encoded2)
decoded2 = Dense(3000, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(decoded2)
decoded2 = Dense(X.shape[1], activation=None,kernel_regularizer=regularizers.l2(0.0005))(decoded2)
autoencoder2 = Model(input_data2, decoded2)
autoencoder2.compile(optimizer=Adam(lr=0.0001), loss=masked_mse)

callback_AE2 = [keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.3, patience=60,verbose=1),\
                    keras.callbacks.ModelCheckpoint(filepath='AE2.h5',monitor='loss',save_best_only=True,\
                                                  save_weights_only=True,mode='min',verbose=1)]

In [30]:
autoencoder2.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 9724)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 3000)              29175000  
_________________________________________________________________
dense_14 (Dense)             (None, 1000)              3001000   
_________________________________________________________________
dense_15 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_16 (Dense)             (None, 1000)              501000    
_________________________________________________________________
dense_17 (Dense)             (None, 3000)              3003000   
_________________________________________________________________
dense_18 (Dense)             (None, 9724)              2918

In [33]:
historyAutoEnc2 = autoencoder2.fit(X, X,
                epochs=80,
                batch_size=256,
                callbacks=callback_AE2,
                shuffle=True)

Epoch 1/80
610/610 [==============================] - 6s 9ms/step - loss: 130.3216

Epoch 00001: loss improved from inf to 130.32157, saving model to AE2.h5
Epoch 2/80
610/610 [==============================] - 5s 8ms/step - loss: 26.3103

Epoch 00002: loss improved from 130.32157 to 26.31032, saving model to AE2.h5
Epoch 3/80
610/610 [==============================] - 5s 8ms/step - loss: 15.7085

Epoch 00003: loss improved from 26.31032 to 15.70854, saving model to AE2.h5
Epoch 4/80
610/610 [==============================] - 5s 8ms/step - loss: 12.4160

Epoch 00004: loss improved from 15.70854 to 12.41603, saving model to AE2.h5
Epoch 5/80
610/610 [==============================] - 5s 8ms/step - loss: 10.4513

Epoch 00005: loss improved from 12.41603 to 10.45128, saving model to AE2.h5
Epoch 6/80
610/610 [==============================] - 5s 8ms/step - loss: 9.4060

Epoch 00006: loss improved from 10.45128 to 9.40602, saving model to AE2.h5
Epoch 7/80
610/610 [========================

Epoch 53/80
610/610 [==============================] - 7s 12ms/step - loss: 5.6143

Epoch 00053: loss improved from 5.64228 to 5.61428, saving model to AE2.h5
Epoch 54/80
610/610 [==============================] - 7s 12ms/step - loss: 5.6053

Epoch 00054: loss improved from 5.61428 to 5.60534, saving model to AE2.h5
Epoch 55/80
610/610 [==============================] - 7s 11ms/step - loss: 5.5833

Epoch 00055: loss improved from 5.60534 to 5.58334, saving model to AE2.h5
Epoch 56/80
610/610 [==============================] - 8s 13ms/step - loss: 5.5706

Epoch 00056: loss improved from 5.58334 to 5.57057, saving model to AE2.h5
Epoch 57/80
610/610 [==============================] - 7s 11ms/step - loss: 5.5543

Epoch 00057: loss improved from 5.57057 to 5.55432, saving model to AE2.h5
Epoch 58/80
610/610 [==============================] - 6s 10ms/step - loss: 5.5277

Epoch 00058: loss improved from 5.55432 to 5.52769, saving model to AE2.h5
Epoch 59/80
610/610 [=========================

In [34]:
X_recons2 = autoencoder2.predict(X)
X_recons2=pd.DataFrame(X_recons2, index=X.index, columns=X.columns)

In [35]:
X_recons2.head(10)

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
userId,,,,,,,,,,,,,,,,,,,,,
1,4.411538,3.698722,4.120959,4.078008,4.481672,3.224706,3.956497,4.410619,3.026774,3.477414,...,4.075370,3.403804,3.632999,4.096186,3.792896,1.869583,4.449799,2.335537,3.225767,3.278914
2,4.183192,3.674538,4.564868,4.362488,4.593633,3.316536,2.970483,4.326844,2.852153,2.720915,...,4.163980,2.797566,3.314810,3.479474,3.403071,1.985072,4.993842,3.030851,3.207874,3.573582
3,4.164542,3.139140,4.474773,4.167884,4.441751,3.213733,3.130360,4.437705,2.855449,2.269766,...,3.997911,2.917126,3.262320,3.404176,3.320507,1.899402,4.521836,3.084283,3.281426,3.239268
4,3.895457,3.839870,3.958004,4.506064,4.874383,3.363368,3.228642,4.120148,2.812693,2.039410,...,3.930277,2.784670,2.967220,3.414876,3.646546,1.814619,4.429069,2.806796,3.484683,3.427606
5,3.843580,3.503752,4.312304,4.366764,4.623276,3.594949,3.250029,4.616611,3.132190,2.502536,...,4.200116,3.027454,3.256617,3.532332,3.681830,1.914356,4.582067,2.968211,3.508910,3.626101
6,3.848145,3.633185,3.592341,5.147496,3.741837,4.321060,4.068096,4.825478,4.067752,1.924535,...,4.160221,2.983373,3.311266,3.463262,3.857177,2.190357,4.330399,2.731030,3.655943,3.319968
7,3.677209,3.494394,4.129771,4.201440,4.449716,3.599779,2.737486,4.079622,2.505783,2.799198,...,4.039425,2.839803,3.058170,3.327354,3.654957,2.061399,4.393517,2.543744,3.539167,3.264165
8,3.920394,3.655381,4.310474,4.386043,4.573958,3.741158,3.475846,4.563460,3.473147,2.562141,...,4.077235,2.908394,3.262815,3.514354,3.692635,1.936638,4.756606,2.957677,3.480976,3.780263
9,3.979017,3.806345,4.354749,4.384103,4.514771,3.591134,3.229727,4.221786,2.889245,2.425347,...,4.145833,2.830258,3.260864,3.637884,3.571350,1.960053,4.741757,2.828782,3.446386,3.625155


### Auto Encoder 2

In [37]:
ohe_userDF = to_categorical(X.index)

In [46]:
input_data3 = Input(shape=(ohe_userDF.shape[1],))
encoded3 = Dense(500, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(input_data3)
decoded3 = Dense(X.shape[1], activation=None,kernel_regularizer=regularizers.l2(0.0005))(encoded3)
autoen3 = Model(input_data3, decoded3)
autoen3.compile(optimizer=Adam(lr=0.0001), loss=masked_mse)

callback_AE3 = [keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.3, patience=60,verbose=1),\
                    keras.callbacks.ModelCheckpoint(filepath='AE3.h5',monitor='loss',save_best_only=True,\
                                                  save_weights_only=True,mode='min',verbose=1)]

In [48]:
autoen3.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 611)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 500)               306000    
_________________________________________________________________
dense_22 (Dense)             (None, 9724)              4871724   
Total params: 5,177,724
Trainable params: 5,177,724
Non-trainable params: 0
_________________________________________________________________


In [51]:
historyAE3 = autoen3.fit(ohe_userDF, X,
                epochs=1000,
                batch_size=256,
                callbacks=callback_AE3,
                shuffle=True)

Epoch 1/1000
610/610 [==============================] - 1s 984us/step - loss: 15.2643

Epoch 00001: loss improved from inf to 15.26433, saving model to AE3.h5
Epoch 2/1000
610/610 [==============================] - 0s 805us/step - loss: 15.2302

Epoch 00002: loss improved from 15.26433 to 15.23016, saving model to AE3.h5
Epoch 3/1000
610/610 [==============================] - 0s 805us/step - loss: 15.1964

Epoch 00003: loss improved from 15.23016 to 15.19640, saving model to AE3.h5
Epoch 4/1000
610/610 [==============================] - 0s 804us/step - loss: 15.1630

Epoch 00004: loss improved from 15.19640 to 15.16302, saving model to AE3.h5
Epoch 5/1000
610/610 [==============================] - 1s 821us/step - loss: 15.1298

Epoch 00005: loss improved from 15.16302 to 15.12981, saving model to AE3.h5
Epoch 6/1000
610/610 [==============================] - 0s 800us/step - loss: 15.0970

Epoch 00006: loss improved from 15.12981 to 15.09703, saving model to AE3.h5
Epoch 7/1000
610/610 

Epoch 51/1000
610/610 [==============================] - 1s 908us/step - loss: 13.2710

Epoch 00051: loss improved from 13.33013 to 13.27102, saving model to AE3.h5
Epoch 52/1000
610/610 [==============================] - 1s 874us/step - loss: 13.2110

Epoch 00052: loss improved from 13.27102 to 13.21101, saving model to AE3.h5
Epoch 53/1000
610/610 [==============================] - 1s 915us/step - loss: 13.1497

Epoch 00053: loss improved from 13.21101 to 13.14974, saving model to AE3.h5
Epoch 54/1000
610/610 [==============================] - 1s 898us/step - loss: 13.0871

Epoch 00054: loss improved from 13.14974 to 13.08713, saving model to AE3.h5
Epoch 55/1000
610/610 [==============================] - 1s 988us/step - loss: 13.0232

Epoch 00055: loss improved from 13.08713 to 13.02318, saving model to AE3.h5
Epoch 56/1000
610/610 [==============================] - 1s 935us/step - loss: 12.9588

Epoch 00056: loss improved from 13.02318 to 12.95880, saving model to AE3.h5
Epoch 57/1

Epoch 152/1000
610/610 [==============================] - 1s 925us/step - loss: 4.8471

Epoch 00152: loss improved from 4.89642 to 4.84712, saving model to AE3.h5
Epoch 153/1000
610/610 [==============================] - 1s 918us/step - loss: 4.7992

Epoch 00153: loss improved from 4.84712 to 4.79925, saving model to AE3.h5
Epoch 154/1000
610/610 [==============================] - 1s 914us/step - loss: 4.7520

Epoch 00154: loss improved from 4.79925 to 4.75202, saving model to AE3.h5
Epoch 155/1000
610/610 [==============================] - 1s 977us/step - loss: 4.7062

Epoch 00155: loss improved from 4.75202 to 4.70621, saving model to AE3.h5
Epoch 156/1000
610/610 [==============================] - 1s 1ms/step - loss: 4.6618

Epoch 00156: loss improved from 4.70621 to 4.66181, saving model to AE3.h5
Epoch 157/1000
610/610 [==============================] - 1s 974us/step - loss: 4.6185

Epoch 00157: loss improved from 4.66181 to 4.61849, saving model to AE3.h5
Epoch 158/1000
610/610 [

Epoch 203/1000
610/610 [==============================] - 1s 1ms/step - loss: 3.5509

Epoch 00203: loss improved from 3.56125 to 3.55088, saving model to AE3.h5
Epoch 204/1000
610/610 [==============================] - 1s 1ms/step - loss: 3.5408

Epoch 00204: loss improved from 3.55088 to 3.54082, saving model to AE3.h5
Epoch 205/1000
610/610 [==============================] - 1s 1ms/step - loss: 3.5308

Epoch 00205: loss improved from 3.54082 to 3.53084, saving model to AE3.h5
Epoch 206/1000
610/610 [==============================] - 1s 1ms/step - loss: 3.5212

Epoch 00206: loss improved from 3.53084 to 3.52121, saving model to AE3.h5
Epoch 207/1000
610/610 [==============================] - 1s 1ms/step - loss: 3.5119

Epoch 00207: loss improved from 3.52121 to 3.51187, saving model to AE3.h5
Epoch 208/1000
610/610 [==============================] - 1s 1ms/step - loss: 3.5029

Epoch 00208: loss improved from 3.51187 to 3.50288, saving model to AE3.h5
Epoch 209/1000
610/610 [==========

Epoch 254/1000
610/610 [==============================] - 1s 2ms/step - loss: 3.2482

Epoch 00254: loss improved from 3.25199 to 3.24817, saving model to AE3.h5
Epoch 255/1000
610/610 [==============================] - 1s 2ms/step - loss: 3.2444

Epoch 00255: loss improved from 3.24817 to 3.24439, saving model to AE3.h5
Epoch 256/1000
610/610 [==============================] - 2s 3ms/step - loss: 3.2406

Epoch 00256: loss improved from 3.24439 to 3.24060, saving model to AE3.h5
Epoch 257/1000
610/610 [==============================] - 2s 3ms/step - loss: 3.2369

Epoch 00257: loss improved from 3.24060 to 3.23688, saving model to AE3.h5
Epoch 258/1000
610/610 [==============================] - 2s 3ms/step - loss: 3.2333

Epoch 00258: loss improved from 3.23688 to 3.23325, saving model to AE3.h5
Epoch 259/1000
610/610 [==============================] - 2s 3ms/step - loss: 3.2296

Epoch 00259: loss improved from 3.23325 to 3.22958, saving model to AE3.h5
Epoch 260/1000
610/610 [==========

...

610/610 [==============================] - 1s 973us/step - loss: 2.2164

Epoch 00762: loss improved from 2.21774 to 2.21638, saving model to AE3.h5
Epoch 763/1000
610/610 [==============================] - 1s 978us/step - loss: 2.2150

Epoch 00763: loss improved from 2.21638 to 2.21504, saving model to AE3.h5
Epoch 764/1000
610/610 [==============================] - 1s 986us/step - loss: 2.2137

Epoch 00764: loss improved from 2.21504 to 2.21370, saving model to AE3.h5
Epoch 765/1000
610/610 [==============================] - 1s 970us/step - loss: 2.2124

Epoch 00765: loss improved from 2.21370 to 2.21236, saving model to AE3.h5
Epoch 766/1000
610/610 [==============================] - 1s 980us/step - loss: 2.2110

Epoch 00766: loss improved from 2.21236 to 2.21101, saving model to AE3.h5
Epoch 767/1000
610/610 [==============================] - 1s 986us/step - loss: 2.2097

Epoch 00767: loss improved from 2.21101 to 2.20965, saving model to AE3.h5
Epoch 768/1000
610/610 [=============

Epoch 813/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.1494

Epoch 00813: loss improved from 2.15067 to 2.14942, saving model to AE3.h5
Epoch 814/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.1481

Epoch 00814: loss improved from 2.14942 to 2.14810, saving model to AE3.h5
Epoch 815/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.1468

Epoch 00815: loss improved from 2.14810 to 2.14679, saving model to AE3.h5
Epoch 816/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.1456

Epoch 00816: loss improved from 2.14679 to 2.14556, saving model to AE3.h5
Epoch 817/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.1443

Epoch 00817: loss improved from 2.14556 to 2.14427, saving model to AE3.h5
Epoch 818/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.1430

Epoch 00818: loss improved from 2.14427 to 2.14298, saving model to AE3.h5
Epoch 819/1000
610/610 [==========

Epoch 864/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0861

Epoch 00864: loss improved from 2.08734 to 2.08610, saving model to AE3.h5
Epoch 865/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0849

Epoch 00865: loss improved from 2.08610 to 2.08495, saving model to AE3.h5
Epoch 866/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0837

Epoch 00866: loss improved from 2.08495 to 2.08371, saving model to AE3.h5
Epoch 867/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0825

Epoch 00867: loss improved from 2.08371 to 2.08251, saving model to AE3.h5
Epoch 868/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0812

Epoch 00868: loss improved from 2.08251 to 2.08123, saving model to AE3.h5
Epoch 869/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0801

Epoch 00869: loss improved from 2.08123 to 2.08010, saving model to AE3.h5
Epoch 870/1000
610/610 [==========

Epoch 915/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0262

Epoch 00915: loss improved from 2.02726 to 2.02616, saving model to AE3.h5
Epoch 916/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0250

Epoch 00916: loss improved from 2.02616 to 2.02505, saving model to AE3.h5
Epoch 917/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0239

Epoch 00917: loss improved from 2.02505 to 2.02386, saving model to AE3.h5
Epoch 918/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0227

Epoch 00918: loss improved from 2.02386 to 2.02270, saving model to AE3.h5
Epoch 919/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0215

Epoch 00919: loss improved from 2.02270 to 2.02149, saving model to AE3.h5
Epoch 920/1000
610/610 [==============================] - 1s 1ms/step - loss: 2.0204

Epoch 00920: loss improved from 2.02149 to 2.02038, saving model to AE3.h5
Epoch 921/1000
610/610 [==========

Epoch 966/1000
610/610 [==============================] - 1s 1ms/step - loss: 1.9694

Epoch 00966: loss improved from 1.97042 to 1.96936, saving model to AE3.h5
Epoch 967/1000
610/610 [==============================] - 1s 1ms/step - loss: 1.9683

Epoch 00967: loss improved from 1.96936 to 1.96831, saving model to AE3.h5
Epoch 968/1000
610/610 [==============================] - 1s 1ms/step - loss: 1.9672

Epoch 00968: loss improved from 1.96831 to 1.96722, saving model to AE3.h5
Epoch 969/1000
610/610 [==============================] - 1s 1ms/step - loss: 1.9662

Epoch 00969: loss improved from 1.96722 to 1.96619, saving model to AE3.h5
Epoch 970/1000
610/610 [==============================] - 1s 1ms/step - loss: 1.9651

Epoch 00970: loss improved from 1.96619 to 1.96507, saving model to AE3.h5
Epoch 971/1000
610/610 [==============================] - 1s 1ms/step - loss: 1.9640

Epoch 00971: loss improved from 1.96507 to 1.96403, saving model to AE3.h5
Epoch 972/1000
610/610 [==========

In [52]:
X_recons3 = autoen3.predict(ohe_userDF)
X_recons3=pd.DataFrame(X_recons3, index=X.index, columns=X.columns)

In [53]:
X_recons3.head(10)

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
userId,,,,,,,,,,,,,,,,,,,,,
1,4.607552,3.689917,4.628265,4.686128,4.966478,3.822425,3.320748,4.809953,3.794938,2.311232,...,0.335472,0.368237,0.368170,0.368247,0.368155,0.302803,0.433887,0.335481,0.368247,0.368254
2,4.023507,3.226012,4.041753,4.089318,4.330281,3.343119,2.911919,4.204477,3.313859,2.036267,...,0.312163,0.340772,0.340707,0.340781,0.340691,0.283601,0.398062,0.312171,0.340785,0.340790
3,3.387537,2.720408,3.405671,3.447457,3.651194,2.823141,2.471963,3.540630,2.793353,1.741593,...,0.286981,0.311099,0.311035,0.311106,0.311020,0.262856,0.359360,0.286989,0.311114,0.311119
4,3.959603,3.174877,3.977685,4.023958,4.263123,3.292044,2.868527,4.131839,3.262222,2.007075,...,0.309669,0.337832,0.337767,0.337840,0.337751,0.281547,0.394228,0.309677,0.337844,0.337850
5,3.824878,3.062985,3.839561,3.886199,4.112923,3.177631,2.771539,3.986922,3.148844,1.942276,...,0.304122,0.331296,0.331230,0.331304,0.331214,0.276977,0.385702,0.304131,0.331308,0.331314
6,4.175374,3.343894,4.191683,4.241263,4.490646,3.464494,3.015527,4.356456,3.436421,2.105386,...,0.318095,0.347762,0.347696,0.347770,0.347680,0.288488,0.407179,0.318104,0.347773,0.347780
7,3.786721,3.035736,3.804953,3.846415,4.075790,3.148334,2.747311,3.951338,3.119746,1.926096,...,0.302726,0.329651,0.329586,0.329659,0.329571,0.275828,0.383556,0.302734,0.329664,0.329670
8,3.816984,3.059377,3.834716,3.878398,4.114791,3.173047,2.767372,3.974890,3.144523,1.939723,...,0.303870,0.330998,0.330934,0.331007,0.330918,0.276770,0.385314,0.303878,0.331011,0.331017
9,4.250338,3.404004,4.266387,4.319854,4.575082,3.526984,3.068254,4.435128,3.496711,2.140589,...,0.321100,0.351304,0.351238,0.351312,0.351222,0.290965,0.411799,0.321109,0.351314,0.351321


### Deep Auto Encoder 2

In [60]:
input_data4 = Input(shape=(ohe_userDF.shape[1],))
encoded4 = Dense(500, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(input_data4)
encoded4 = Dense(400, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(encoded4)
encoded4 = Dense(300, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(encoded4)
decoded4 = Dense(400, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(encoded4)
decoded4 = Dense(500, activation='relu',kernel_regularizer=regularizers.l2(0.0005))(decoded4)
decoded4 = Dense(9724, activation=None,kernel_regularizer=regularizers.l2(0.0005))(decoded4)
autoen4 = Model(input_data4, decoded4)
autoen4.compile(optimizer=Adam(lr=0.0001), loss=masked_mse)

callback_AE4 = [keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.3, patience=60,verbose=1),\
                    keras.callbacks.ModelCheckpoint(filepath='AE4.h5',monitor='loss',save_best_only=True,\
                                                  save_weights_only=True,mode='min',verbose=1)]

In [62]:
autoen4.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 611)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 500)               306000    
_________________________________________________________________
dense_30 (Dense)             (None, 400)               200400    
_________________________________________________________________
dense_31 (Dense)             (None, 300)               120300    
_________________________________________________________________
dense_32 (Dense)             (None, 400)               120400    
_________________________________________________________________
dense_33 (Dense)             (None, 500)               200500    
_________________________________________________________________
dense_34 (Dense)             (None, 9724)              4871

In [64]:
historyAE4 = autoen4.fit(ohe_userDF, X,
                epochs=80,
                batch_size=256,
                callbacks=callback_AE4,
                shuffle=True)

Epoch 1/80
610/610 [==============================] - 1s 1ms/step - loss: 16.0478

Epoch 00001: loss improved from inf to 16.04782, saving model to AE4.h5
Epoch 2/80
610/610 [==============================] - 1s 920us/step - loss: 16.0161

Epoch 00002: loss improved from 16.04782 to 16.01611, saving model to AE4.h5
Epoch 3/80
610/610 [==============================] - 1s 909us/step - loss: 15.9832

Epoch 00003: loss improved from 16.01611 to 15.98324, saving model to AE4.h5
Epoch 4/80
610/610 [==============================] - 1s 937us/step - loss: 15.9488

Epoch 00004: loss improved from 15.98324 to 15.94876, saving model to AE4.h5
Epoch 5/80
610/610 [==============================] - 1s 940us/step - loss: 15.9113

Epoch 00005: loss improved from 15.94876 to 15.91131, saving model to AE4.h5
Epoch 6/80
610/610 [==============================] - 1s 958us/step - loss: 15.8694

Epoch 00006: loss improved from 15.91131 to 15.86943, saving model to AE4.h5
Epoch 7/80
610/610 [===============

In [65]:
X_recons4 = autoen4.predict(ohe_userDF)
X_recons4=pd.DataFrame(X_recons4, index=X.index, columns=X.columns)

In [66]:
X_recons4.head(10)

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
userId,,,,,,,,,,,,,,,,,,,,,
1,4.638010,3.714369,4.680544,4.691593,4.961983,3.997417,3.960366,4.755718,3.882052,3.409981,...,2.200220,2.584808,2.544264,2.541918,2.553986,1.821564,3.320712,2.195111,2.577833,2.591186
2,3.977834,3.186589,4.011627,4.015517,4.252854,3.433338,3.391634,4.082362,3.329649,2.924911,...,1.887070,2.215246,2.182497,2.179462,2.189700,1.562764,2.848705,1.883347,2.209105,2.221611
3,2.968063,2.380910,2.995224,2.998890,3.179501,2.561961,2.531719,3.046940,2.486133,2.184329,...,1.410767,1.656112,1.632710,1.630091,1.636830,1.168864,2.127472,1.407727,1.653916,1.660946
4,3.774565,3.030362,3.811232,3.821386,4.042195,3.258654,3.216192,3.877708,3.163870,2.776477,...,1.793621,2.104867,2.074234,2.070899,2.081304,1.485745,2.705405,1.788196,2.101159,2.109316
5,3.878465,3.114839,3.916283,3.921304,4.150431,3.343551,3.308398,3.982362,3.250566,2.858620,...,1.841366,2.161541,2.129411,2.128253,2.137396,1.524830,2.778839,1.836938,2.156399,2.165671
6,4.173041,3.347321,4.216356,4.221219,4.471398,3.604546,3.564301,4.285089,3.500688,3.074687,...,1.982913,2.328192,2.291587,2.290645,2.300285,1.642155,2.993495,1.978686,2.323646,2.332732
7,3.725221,2.989236,3.765880,3.764361,3.977945,3.215921,3.175694,3.831627,3.120408,2.742798,...,1.767736,2.075345,2.045389,2.043171,2.052918,1.464126,2.667939,1.764654,2.071503,2.079839
8,3.934693,3.155153,3.972757,3.978503,4.212896,3.393260,3.355434,4.034346,3.295818,2.891342,...,1.868300,2.195000,2.160989,2.157144,2.169381,1.548075,2.819299,1.863805,2.190401,2.199110
9,4.092886,3.284223,4.136097,4.139823,4.384928,3.535615,3.493329,4.207545,3.432308,3.012965,...,1.941458,2.280946,2.246745,2.245346,2.255952,1.608861,2.932418,1.939220,2.276884,2.286335
